In [4]:
import pandas as pd
import featuretools as ft
# 使用pandas_profiling库分析数据情况
# import pandas_profiling
# train_df.profile_report(title='dc营销')

## data_info


 
### 特征加工时间窗口:
方案一： 训练：Q3 + Q4   预测： Q4+Q1  


方案二： 数据窗口训练：Q3 | Q4  预测：Q1 方案较为简单。


###  整体思路

 识别意向客户的一个场景，最终抽象成一个分类问题（标签有-1 0 1）；
 
 数据：基本信息、存款 资产、行为信息。训练的时间窗口有两个季度，每个季度都有相应y标签；测试集是第三个季度；
 
 特征衍生： 01基础特征；03.由字段含义衍生；（12.序列预测作为特征；考虑计算资源 无加工） 04.多轮迭代重要特征细加工Featuretools（考虑计算资源，只挑部分的重要特征）；
 
 特重要性选择： 缺失率、单值率、方差、相关性； gain
 
 模型选择：lgb+bayes优化
 
 模型融合：样本随机抽样+参数扰动 子模型模型 --》均值融合 ，（ LR_blending ..）
 


In [5]:
# 交集情况
y_Q3_3 = pd.read_csv('data/y_train/y_Q3_3.csv')
y_Q4_3 = pd.read_csv('data/y_train/y_Q4_3.csv')

In [6]:
# # 验证train集的预测结果
"""
 1.0    48601
 0.0    14668
-1.0     7003
 NaN     6450
"""
# cust_avli_Q1 = pd.read_csv('data/x_test/cust_avli_Q1.csv',index_col=0)
# result = pd.read_csv('data/y_train/y_Q4_3.csv',index_col=0) # train result concat
# cust_avli_Q1.merge(result,how='left',right_index=True,left_index=True).fillna(1).drop('label_x',axis=1).to_csv('data/x_test/result4.csv')
# cust_avli_Q1.merge(result,how='left',right_index=True,left_index=True).drop('label_x',axis=1).label_y.value_counts(dropna=False)

'\n 1.0    48601\n 0.0    14668\n-1.0     7003\n NaN     6450\n'

 ### 方案一

In [7]:
# Q3 aum
aum_m7 = pd.read_csv('data/x_train/aum_m7.csv',index_col=0)
aum_m8 = pd.read_csv('data/x_train/aum_m8.csv',index_col=0)
aum_m9 = pd.read_csv('data/x_train/aum_m9.csv',index_col=0)
print(aum_m7.shape)
print(len(aum_m7.index.unique()))
aum_m7.head()

# Q4 aum
aum_m10 = pd.read_csv('data/x_train/aum_m10.csv',index_col=0)
aum_m11 = pd.read_csv('data/x_train/aum_m11.csv',index_col=0)
aum_m12 = pd.read_csv('data/x_train/aum_m12.csv',index_col=0)
print(aum_m11.shape)
print(len(aum_m11.index.unique()))
aum_m11.head()

# Q1 aum
aum_m1 = pd.read_csv('data/x_test/aum_m1.csv',index_col=0)
aum_m2 = pd.read_csv('data/x_test/aum_m2.csv',index_col=0)
aum_m3 = pd.read_csv('data/x_test/aum_m3.csv',index_col=0)
print(aum_m1.shape)
print(len(aum_m1.index.unique()))
aum_m1.head()

(465441, 8)
465441
(521566, 8)
521566
(556195, 8)
556195


,X1,X2,X3,X4,X5,X6,X7,X8
cust_no,,,,,,,,
0xb2d76a72,0,0.0,0.00,0,0.00,0.0,0.0,0
0xb2d146f9,0,0.0,0.00,0,0.00,0.0,0.0,0
0xb2d281e4,200000,0.0,2.31,0,160.41,0.0,0.0,0
0xb2d73142,0,0.0,0.00,0,0.00,0.0,0.0,0
0xb2d70a75,0,0.0,0.00,0,0.00,0.0,0.0,0


In [8]:
# Q3 cunkuan

cunkuan_m7 = pd.read_csv('data/x_train/cunkuan_m7.csv',index_col=0)
cunkuan_m8 = pd.read_csv('data/x_train/cunkuan_m8.csv',index_col=0)
cunkuan_m9 = pd.read_csv('data/x_train/cunkuan_m9.csv',index_col=0)
print(cunkuan_m7.shape)
print(len(cunkuan_m7.index.unique()))
cunkuan_m7.head()

# Q4 cunkuan

cunkuan_m10 = pd.read_csv('data/x_train/cunkuan_m10.csv',index_col=0)
cunkuan_m11 = pd.read_csv('data/x_train/cunkuan_m11.csv',index_col=0)
cunkuan_m12 = pd.read_csv('data/x_train/cunkuan_m12.csv',index_col=0)
print(cunkuan_m10.shape)
print(len(cunkuan_m10.index.unique()))
cunkuan_m10.head()

# Q1 cunkuan

cunkuan_m1 = pd.read_csv('data/x_test/cunkuan_m1.csv',index_col=0)
cunkuan_m2 = pd.read_csv('data/x_test/cunkuan_m2.csv',index_col=0)
cunkuan_m3 = pd.read_csv('data/x_test/cunkuan_m3.csv',index_col=0)
print(cunkuan_m1.shape)
print(len(cunkuan_m1.index.unique()))
cunkuan_m1.head()

(181220, 2)
181220
(209862, 2)
209862
(245490, 2)
245490


,C1,C2
cust_no,,
0xb2d636dd,0.31,1
0xb2d636da,600983.91,7
0xb2d3d205,163.24,1
0xb2d3d20b,40000.18,3
0xb2d40f1f,0.00,1


In [9]:
# Q3 behavior.csv
behavior_m7 = pd.read_csv('data/x_train/behavior_m7.csv',index_col=0)
behavior_m8 = pd.read_csv('data/x_train/behavior_m8.csv',index_col=0)
behavior_m9 = pd.read_csv('data/x_train/behavior_m9.csv',index_col=0)
print(behavior_m7.shape)
print(len(behavior_m7.index.unique()))
behavior_m7.head()

# Q4 behavior.csv
behavior_m10 = pd.read_csv('data/x_train/behavior_m10.csv',index_col=0)
behavior_m11 = pd.read_csv('data/x_train/behavior_m11.csv',index_col=0)
behavior_m12 = pd.read_csv('data/x_train/behavior_m12.csv',index_col=0)
print(behavior_m10.shape)
print(len(behavior_m10.index.unique()))
behavior_m10.head()


# Q1 behavior.csv
behavior_m1 = pd.read_csv('data/x_test/behavior_m1.csv',index_col=0)
behavior_m2 = pd.read_csv('data/x_test/behavior_m2.csv',index_col=0)
behavior_m3 = pd.read_csv('data/x_test/behavior_m3.csv',index_col=0)
print(behavior_m1.shape)
print(len(behavior_m1.index.unique()))
behavior_m1.head()

(465441, 5)
465441
(506513, 5)
506513
(556195, 5)
556195


,B1,B2,B3,B4,B5
cust_no,,,,,
0xb2d6beaa,0,1,4600.00,0,0.0
0xb2d8395e,4,1,1260.00,0,0.0
0xb2d80715,0,0,0.00,0,0.0
0xb2d9383d,56,4,169945.73,6,170601.0
0xb2d77256,0,0,0.00,0,0.0


In [10]:
behavior_m9.head()

,B1,B2,B3,B4,B5,B6,B7
cust_no,,,,,,,
0xb2d5e9bd,0,0,0.0,0,0.0,2019-09-27 23:07:00,23
0xb2d5e90a,0,1,1539000.0,2,1000000.0,2019-09-19 10:45:00,7
0xb2d6be3b,0,0,0.0,0,0.0,NaN,0
0xb2d0cb57,0,0,0.0,0,0.0,NaN,0
0xb2d550c0,0,0,0.0,0,0.0,NaN,0


In [11]:
#Q3 big_event
big_event_Q3 = pd.read_csv('data/x_train/big_event_Q3.csv',index_col=0)
print(big_event_Q3.shape)
print(len(big_event_Q3.index.unique()))
big_event_Q3.head()

#Q4 big_event
big_event_Q4 = pd.read_csv('data/x_train/big_event_Q4.csv',index_col=0)
print(big_event_Q4.shape)
print(len(big_event_Q4.index.unique()))
big_event_Q4.head()

#Q1 big_event
big_event_Q1 = pd.read_csv('data/x_test/big_event_Q1.csv',index_col=0)
print(big_event_Q1.shape)
print(len(big_event_Q1.index.unique()))
big_event_Q1.head()
# pd.to_datetime(cust_df.E1) - pd.to_datetime(cust_df.E2)

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(493441, 18)
493441
(543823, 18)
543823
(659624, 18)
659624


,E1,E2,E3,E4,E5,E6,E7,E8,E9,E10,E11,E12,E13,E14,E15,E16,E17,E18
cust_no,,,,,,,,,,,,,,,,,,
0xb2d1c438,2016-05-15,2016-05-15,2016-05-15,NaN,NaN,2016-08-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
0xb2dae685,2020-02-18,2020-02-18,2020-02-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
0xb2d2e22d,2017-06-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
0xb2d1f94a,2016-07-05,2016-07-05,2016-07-05,NaN,2016-07-05,NaN,NaN,NaN,NaN,2020-03-20,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN
0xb2d8d469,2019-07-09,2019-07-09,2019-07-09,NaN,2019-07-10,2019-07-10,NaN,NaN,NaN,2019-07-10,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN


In [12]:
# Q3 cust_info.csv

cust_info_q3 = pd.read_csv('data/x_train/cust_info_q3.csv',index_col=0)
print(cust_info_q3.shape)
print(len(cust_info_q3.index.unique()))
cust_info_q3.head()

# Q4 cust_info.csv

cust_info_q4 = pd.read_csv('data/x_train/cust_info_q4.csv',index_col=0)
print(cust_info_q4.shape)
print(len(cust_info_q4.index.unique()))
cust_info_q4.head()

# Q1 cust_info.csv

cust_info_q1 = pd.read_csv('data/x_test/cust_info_q1.csv',index_col=0)
print(cust_info_q1.shape)
print(len(cust_info_q1.index.unique()))
cust_info_q1.head()

(493441, 20)
493441
(543823, 20)
543823
(659624, 20)
659624


,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,I14,I15,I16,I17,I18,I19,I20
cust_no,,,,,,,,,,,,,,,,,,,,
0xb2d5fd8e,男性,26,普通客户,0.0,商业工作人员,0,0,白羊座,NaN,NaN,0.0,个人,NaN,NaN,0,0.0,1,1,1,0
0xb2d811d2,男性,25,普通客户,0.0,不便分类的其他从业人员,0,0,天蝎座,NaN,未知,0.0,个人,未说明的婚姻状况,未知,0,0.0,0,0,0,0
0xb2d8b1c8,女性,43,普通客户,0.0,商业工作人员,0,0,水瓶座,NaN,NaN,0.0,个人,NaN,NaN,0,0.0,0,0,0,0
0xb2db077a,女性,58,普通客户,0.0,服务性工作人员,0,0,双鱼座,NaN,NaN,0.0,个人,NaN,NaN,0,0.0,1,1,1,0
0xb2da6784,男性,24,普通客户,0.0,服务性工作人员,0,0,双子座,NaN,NaN,0.0,个人,NaN,NaN,0,0.0,1,1,1,0


In [13]:
import pandas as pd
# Q3 cust_avli.csv
cust_avli_Q3 = pd.read_csv('data/x_train/cust_avli_Q3.csv',index_col=0)
# 全连接方法，保留用户的avlib标志位
cust_avli_Q3['valid'] = 1
cust_avli_Q3.head()

# Q4 cust_avli.csv
cust_avli_Q4 = pd.read_csv('data/x_train/cust_avli_Q4.csv',index_col=0)
cust_avli_Q4['valid'] = 1
cust_avli_Q4.head()

# Q3 cust_avli.csv
cust_avli_Q1 = pd.read_csv('data/x_test/cust_avli_Q1.csv',index_col=0,usecols=[0])
cust_avli_Q1['valid'] = 1
cust_avli_Q1.head()


,valid
cust_no,
0x3b9b4615,1
0x3b9ae61b,1
0x3b9add69,1
0x3b9b3601,1
0x3b9b2599,1


In [14]:
# Q3 merge 
# 连接方式left 为有效宽表版本； outer 为全连接版本也关联了无效的用户

q3_df = cust_avli_Q3.merge(
    cust_info_q3, how='outer', right_index=True, left_index=True, suffixes=('','')).merge(
    big_event_Q3, how='outer', right_index=True, left_index=True, suffixes=('','')).merge(
    behavior_m7, how='outer', right_index=True, left_index=True, suffixes=('','_fir')).merge(
    behavior_m8, how='outer', right_index=True, left_index=True, suffixes=('_fir','_sec')).merge(
    behavior_m9, how='outer', right_index=True, left_index=True, suffixes=('_sec','_last')).merge(
    cunkuan_m7, how='outer', right_index=True, left_index=True, suffixes=('','_fir')).merge(
    cunkuan_m8, how='outer', right_index=True, left_index=True, suffixes=('_fir','_sec')).merge(
    cunkuan_m9, how='outer', right_index=True, left_index=True, suffixes=('_sec','_last')).merge(
    aum_m7, how='outer', right_index=True, left_index=True, suffixes=('','_fir')).merge(
    aum_m8, how='outer', right_index=True, left_index=True, suffixes=('_fir','_sec')).merge(
    aum_m9, how='outer', right_index=True, left_index=True, suffixes=('_sec','_last'))




# Q4 merge 
q4_df = cust_avli_Q4.merge(
    cust_info_q4, how='outer', right_index=True, left_index=True, suffixes=('','')).merge(
    big_event_Q4, how='outer', right_index=True, left_index=True, suffixes=('','')).merge(
    behavior_m10, how='outer', right_index=True, left_index=True, suffixes=('','_fir')).merge(
    behavior_m11, how='outer', right_index=True, left_index=True, suffixes=('_fir','_sec')).merge(
    behavior_m12, how='outer', right_index=True, left_index=True, suffixes=('_sec','_last')).merge(
    cunkuan_m10, how='outer', right_index=True, left_index=True, suffixes=('','_fir')).merge(
    cunkuan_m11, how='outer', right_index=True, left_index=True, suffixes=('_fir','_sec')).merge(
    cunkuan_m12, how='outer', right_index=True, left_index=True, suffixes=('_sec','_last')).merge(
    aum_m10, how='outer', right_index=True, left_index=True, suffixes=('','_fir')).merge(
    aum_m11, how='outer', right_index=True, left_index=True, suffixes=('_fir','_sec')).merge(
    aum_m12, how='outer', right_index=True, left_index=True, suffixes=('_sec','_last'))




# Q1 merge 
q1_df = cust_avli_Q1.merge(
    cust_info_q1, how='outer', right_index=True, left_index=True, suffixes=('','')).merge(
    big_event_Q1, how='outer', right_index=True, left_index=True, suffixes=('','')).merge(
    behavior_m1, how='outer', right_index=True, left_index=True, suffixes=('','_fir')).merge(
    behavior_m2, how='outer', right_index=True, left_index=True, suffixes=('_fir','_sec')).merge(
    behavior_m3, how='outer', right_index=True, left_index=True, suffixes=('_sec','_last')).merge(
    cunkuan_m1, how='outer', right_index=True, left_index=True, suffixes=('','_fir')).merge(
    cunkuan_m2, how='outer', right_index=True, left_index=True, suffixes=('_fir','_sec')).merge(
    cunkuan_m3, how='outer', right_index=True, left_index=True, suffixes=('_sec','_last')).merge(
    aum_m1, how='outer', right_index=True, left_index=True, suffixes=('','_fir')).merge(
    aum_m2, how='outer', right_index=True, left_index=True, suffixes=('_fir','_sec')).merge(
    aum_m3, how='outer', right_index=True, left_index=True, suffixes=('_sec','_last'))


In [15]:
print(q3_df.shape)
q3_df.head()

print(q4_df.shape)
q4_df.head()

print(q1_df.shape)
q1_df.head()

(493567, 86)
(544190, 86)
(660183, 86)


,valid,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,X7_sec,X8_sec,X1,X2,X3,X4,X5,X6,X7,X8
cust_no,,,,,,,,,,,,,,,,,,,,,
0x3b9acb25,NaN,女性,55.0,普通客户,0.0,NaN,0.0,0.0,巨蟹座,NaN,...,0.00,0.0,0.0,0.0,7.01,0.0,0.0,0.0,0.00,0.0
0x3b9acb28,NaN,女性,65.0,普通客户,0.0,NaN,0.0,0.0,狮子座,NaN,...,0.00,0.0,0.0,0.0,68.37,0.0,0.0,0.0,0.00,0.0
0x3b9acb2a,NaN,女性,0.0,普通客户,0.0,NaN,0.0,0.0,摩羯座,NaN,...,0.00,0.0,0.0,0.0,8.29,0.0,0.0,0.0,0.00,0.0
0x3b9acb2b,1.0,男性,43.0,普通客户,0.0,服务性工作人员,1.0,0.0,天蝎座,NaN,...,952457.67,0.0,0.0,0.0,1649.54,0.0,0.0,0.0,959874.08,0.0
0x3b9acb2c,NaN,NaN,41.0,普通客户,0.0,NaN,0.0,0.0,金牛座,NaN,...,0.00,0.0,0.0,0.0,0.17,0.0,0.0,0.0,0.00,0.0


In [16]:
# 为样本生成唯一标识

q3_df.reset_index('cust_no', inplace=True)
q4_df.reset_index('cust_no', inplace=True)
q1_df.reset_index('cust_no', inplace=True)

# 拼接 q1 q3 q4
# 生成唯一index
q3_df['cust_q'] = 'q3' + q3_df['cust_no']
q4_df['cust_q'] = 'q4' + q4_df['cust_no']
q1_df['cust_q'] = 'q1' + q1_df['cust_no']

# drop cust
# q3_df.drop(['cust_no'],axis=1, inplace=True)
# q4_df.drop(['cust_no'],axis=1, inplace=True)
# q1_df.drop(['cust_no'],axis=1, inplace=True)
q1_df.head()

,cust_no,valid,I1,I2,I3,I4,I5,I6,I7,I8,...,X8_sec,X1,X2,X3,X4,X5,X6,X7,X8,cust_q
0,0x3b9acb25,NaN,女性,55.0,普通客户,0.0,NaN,0.0,0.0,巨蟹座,...,0.0,0.0,0.0,7.01,0.0,0.0,0.0,0.00,0.0,q10x3b9acb25
1,0x3b9acb28,NaN,女性,65.0,普通客户,0.0,NaN,0.0,0.0,狮子座,...,0.0,0.0,0.0,68.37,0.0,0.0,0.0,0.00,0.0,q10x3b9acb28
2,0x3b9acb2a,NaN,女性,0.0,普通客户,0.0,NaN,0.0,0.0,摩羯座,...,0.0,0.0,0.0,8.29,0.0,0.0,0.0,0.00,0.0,q10x3b9acb2a
3,0x3b9acb2b,1.0,男性,43.0,普通客户,0.0,服务性工作人员,1.0,0.0,天蝎座,...,0.0,0.0,0.0,1649.54,0.0,0.0,0.0,959874.08,0.0,q10x3b9acb2b
4,0x3b9acb2c,NaN,NaN,41.0,普通客户,0.0,NaN,0.0,0.0,金牛座,...,0.0,0.0,0.0,0.17,0.0,0.0,0.0,0.00,0.0,q10x3b9acb2c


In [17]:
cust_df = pd.concat([q3_df, q4_df, q1_df])
print(len(cust_df.cust_q.unique()))
print(len(cust_df.cust_q))

1697940
1697940


In [20]:
# 生成宽表

# 读入y标签
y_Q3_3 = pd.read_csv('data/y_train/y_Q3_3.csv')
y_Q4_3 = pd.read_csv('data/y_train/y_Q4_3.csv')
# 唯一标识
y_Q3_3['cust_q'] = 'q3' + y_Q3_3['cust_no']
y_Q4_3['cust_q'] = 'q4' + y_Q4_3['cust_no']

# drop
y_Q4_3.drop(['cust_no'],axis=1, inplace=True)
y_Q3_3.drop(['cust_no'],axis=1, inplace=True)

# 合并
y_df = pd.concat([y_Q3_3,y_Q4_3])

# 关联 x,y  
# train_df =  cust_df.merge(y_df,how='left',left_on='cust_q',right_on='cust_q')
# train_df.to_pickle('./data/outerjoin_raw_features_label.pkl')
train_df =  cust_df.merge(y_df,how='inner',left_on='cust_q',right_on='cust_q')
train_df.to_pickle('./data/raw_features_label.pkl')
print(cust_df.shape)
print(train_df.shape)
train_df.head()

(1697940, 88)
(145296, 89)


,cust_no,valid,I1,I2,I3,I4,I5,I6,I7,I8,...,X1,X2,X3,X4,X5,X6,X7,X8,cust_q,label
0,0x3b9acb2b,1.0,男性,42.0,普通客户,0.0,NaN,1.0,0.0,天蝎座,...,0.0,0.00,2412.41,0.0,0.0,0.0,997737.81,0.0,q30x3b9acb2b,0
1,0x3b9acc6c,1.0,女性,37.0,普通客户,0.0,NaN,1.0,0.0,天秤座,...,0.0,0.00,1229.22,0.0,0.0,0.0,0.00,0.0,q30x3b9acc6c,1
2,0x3b9acc87,1.0,女性,68.0,黄金,0.0,不便分类的其他从业人员,0.0,0.0,双子座,...,100000.0,0.00,48.51,0.0,0.0,0.0,0.00,0.0,q30x3b9acc87,1
3,0x3b9acca4,1.0,女性,69.0,普通客户,0.0,服务性工作人员,0.0,0.0,处女座,...,0.0,20787.61,12523.42,0.0,0.0,0.0,0.00,0.0,q30x3b9acca4,1
4,0x3b9accb7,1.0,女性,49.0,普通客户,0.0,NaN,1.0,0.0,双子座,...,0.0,0.00,4155.11,0.0,0.0,0.0,0.00,0.0,q30x3b9accb7,1


### 数据分析 

In [21]:
#raw features  to pickle
import pandas as pd
import numpy as np

# train_df.to_pickle('./data/raw_features_label.pkl')

train_df = pd.read_pickle('./data/raw_features_label.pkl')


In [ ]:
# 基础特征分布情况

train_df.label.value_counts(dropna=False)

pd.options.display.max_columns = None
pd.options.display.max_rows = None
np.set_printoptions(threshold=np.inf)
group_df = train_df.groupby('label').agg(['nunique','count'])
group_df.head()